In [1]:
%load_ext autoreload
%autoreload 2

In [60]:
import pandas as pd
import numpy as np
from filtering_functions import *
import pickle as pkl
from tqdm.notebook import tqdm
import json
import matplotlib.pyplot as plt

In [3]:
import os
os.chdir('..')

In [4]:
with open('cache/df_vk2023.pkl', 'rb') as f:
  df = pkl.load(f)

In [5]:
df

,text,vk_wall_id,vk_post_id,attachment_types,n_unique_attachments,has_link
0,Распродажа\nЦена: 150руб.\nАрт: 2В-23/1Б корпус А,-209013890,35997,[photo],0,False
1,Распродажа\nЦена: 250р\nМатеирал: с ночёсом\nР...,-209013890,35998,[photo],0,False
2,Распродажа \nЦена: 500р \nМатеирал: ангора \nР...,-209013890,35999,[photo],0,False
3,,-209013890,36000,[],0,False
4,Распродажа\nЦена: 200руб.\nАрт: 18-82\nРазмеры...,-209013890,36001,[photo],0,False
...,...,...,...,...,...,...
213014,💎Пуховик The North Face💎 \n\n- Материал: Плаще...,-1485,211385,"[photo, link]",1,True
213015,"Багажник для велосипеда передний,новый\n#барах...",-1485,211388,"[photo, link]",1,True
213016,"Джинсы в отличном состоянии, брендовых фирм, р...",-1485,211389,"[photo, link]",1,True
213017,Продаю Sony PS 5 вeрcия c диcкoвoдом ( 3 рeвиз...,-1485,211390,[photo],0,False


In [61]:
stats_to_apply = {
  'is_selling_text': is_selling_text,
  'is_not_selling_text': is_bad_text,
  'count_numbers': count_numbers,
  'count_tokens': count_input_tokens,
  'count_number_sequences': count_number_sequences,
  'count_unique_symbols': count_unique_symbols,
  'text_len': lambda text: len(text),
  'count_words': lambda text: len(text.split()),
}

In [62]:
# add new columns
for name, function in tqdm(stats_to_apply.items()):
  df[name] = df.text.apply(function)

  0%|          | 0/8 [00:00<?, ?it/s]

In [11]:
df.head(5)

,text,vk_wall_id,vk_post_id,attachment_types,n_unique_attachments,has_link,is_selling_text,is_not_selling_text,count_numbers,count_input_tokens,count_number_sequences,count_unique_symbols,text_len,count_words
0,Распродажа\nЦена: 150руб.\nАрт: 2В-23/1Б корпус А,-209013890,35997,[photo],0,False,True,False,4,32,4,2,47,7
1,Распродажа\nЦена: 250р\nМатеирал: с ночёсом\nР...,-209013890,35998,[photo],0,False,True,False,11,85,11,8,148,16
2,Распродажа \nЦена: 500р \nМатеирал: ангора \nР...,-209013890,35999,[photo],0,False,True,False,9,52,9,4,79,9
3,,-209013890,36000,[],0,False,False,False,0,0,0,0,0,0
4,Распродажа\nЦена: 200руб.\nАрт: 18-82\nРазмеры...,-209013890,36001,[photo],0,False,True,False,4,65,8,5,106,14


In [31]:
df['index_info'] = df[['vk_wall_id', 'vk_post_id']].apply(lambda row: json.dumps(row.to_dict()), axis=1) #json.dumps({row}), axis=1)

In [33]:
df = df.drop(columns=['vk_wall_id', 'vk_post_id'])

In [35]:
df.columns

Index(['text', 'attachment_types', 'n_unique_attachments', 'has_link',
       'is_selling_text', 'is_not_selling_text', 'count_numbers',
       'count_input_tokens', 'count_number_sequences', 'count_unique_symbols',
       'text_len', 'count_words', 'index_info'],
      dtype='object')

In [57]:
def apply_filter(df, stat_name, low_limit=None, top_limit=None, eq_val=None):
  mask = pd.Series(np.ones(df.shape[0], dtype=bool), index=df.index)
  if low_limit is not None:
    mask &= df[stat_name] > low_limit
  if top_limit is not None:
    mask &= df[stat_name] < top_limit
  if eq_val is not None:
    mask &= df[stat_name] == eq_val
  return df[mask]

In [59]:
apply_filter(df, 'text_len', 10, 20).text_len

34938     12
35149     12
143473    16
143531    14
143585    19
          ..
208795    16
208797    14
209064    15
209195    18
209229    11
Name: text_len, Length: 831, dtype: int64

In [66]:
df_filtered = apply_filter(df, 'count_tokens', 2, 600)

In [67]:
df_filtered = apply_filter(df_filtered, 'has_link', eq_val=False)

In [68]:
df_filtered = apply_filter(df_filtered, 'is_selling_text', eq_val=True)

In [69]:
df_filtered = apply_filter(df_filtered, 'is_not_selling_text', eq_val=False)

In [70]:
df[df.count_number_sequences != df.count_numbers]

,text,attachment_types,n_unique_attachments,has_link,is_selling_text,is_not_selling_text,count_numbers,count_input_tokens,count_number_sequences,count_unique_symbols,text_len,count_words,index_info,count_tokens
4,Распродажа\nЦена: 200руб.\nАрт: 18-82\nРазмеры...,[photo],0,False,True,False,4,65,8,5,106,14,"{""vk_wall_id"": -209013890, ""vk_post_id"": 36001}",65
11,Распродажа\nКостюм двойка\nЦена: 250руб.\nМате...,[photo],0,False,True,False,4,63,5,6,98,14,"{""vk_wall_id"": -209013890, ""vk_post_id"": 36035}",63
12,Распродажа\nЛосины\nЦена: 99руб.\nМатериал: с ...,[photo],0,False,True,False,4,61,8,5,94,13,"{""vk_wall_id"": -209013890, ""vk_post_id"": 36040}",61
15,Распродажа\nСветильник\nЦена: 150руб.\nАрт: СТ...,[photo],0,False,True,False,5,46,7,4,66,8,"{""vk_wall_id"": -209013890, ""vk_post_id"": 36043}",46
24,Распродажа\nТолстовка\nЦена: 200руб.\nМатериал...,[photo],0,False,True,False,4,61,7,5,92,11,"{""vk_wall_id"": -209013890, ""vk_post_id"": 36052}",61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212969,✅Рено Логан \n2007 \n1.4 бензин \nПробег 250 0...,[photo],0,False,False,False,11,236,12,13,436,61,"{""vk_wall_id"": -1485, ""vk_post_id"": 211178}",236
212977,"✅Hyundai Tucson \n2.0 бензин, АКПП \n2018 год ...",[photo],0,False,False,False,12,296,14,13,568,73,"{""vk_wall_id"": -1485, ""vk_post_id"": 211228}",296
212991,"Ноутбук Samsung r-525\n•••\nЭкран -15.6"" 1366*...","[photo, link]",1,True,False,False,16,357,19,76,622,84,"{""vk_wall_id"": -1485, ""vk_post_id"": 211308}",357
212994,✅Хендай Солярис \n2012 \n1.6 бензин МКПП \nРод...,[photo],0,False,False,False,11,290,12,15,526,67,"{""vk_wall_id"": -1485, ""vk_post_id"": 211313}",290


In [73]:
df.loc[4, 'text']

'Распродажа\nЦена: 200руб.\nАрт: 18-82\nРазмеры: 48,50,52,54,56\nВыбор цвета: без выбора цвета\nМатериал: хлопок'

In [74]:
df_filtered = apply_filter(df_filtered, 'count_number_sequences', low_limit=1, top_limit=40)
df_filtered = apply_filter(df_filtered, 'count_numbers', low_limit=1, top_limit=35)

In [75]:
df_filtered = apply_filter(df_filtered, 'count_unique_symbols', top_limit=30)

In [77]:
df_filtered = apply_filter(df_filtered, 'text_len', top_limit=1000)

In [79]:
df_filtered.shape

(123238, 14)

In [80]:
df_filtered.to_pickle('cache/vk_df_filtered.pkl')

In [83]:
RANDOM_STATE = 42

df_filtered_sample = df_filtered.sample(n=1500, random_state=RANDOM_STATE)[['text', 'index_info']]

In [84]:
df_filtered_sample.shape

(1500, 2)

In [85]:
df_filtered_sample.head(5)

,text,index_info
48116,Распродажа \nСережки \nЦена: 60р\nУпк от 10шт ...,"{""vk_wall_id"": -206817857, ""vk_post_id"": 75889}"
98375,Распродажа \nЦена: 150р \nЦена уп 5×140р\nРазм...,"{""vk_wall_id"": -203393330, ""vk_post_id"": 103913}"
48348,Распродажа\nРост:86-92-98-104-110-116\nЦена: 5...,"{""vk_wall_id"": -206817857, ""vk_post_id"": 76318}"
104934,Распродажа \nКрюч \nЦена:35р\nАрт:9-46,"{""vk_wall_id"": -203393330, ""vk_post_id"": 116860}"
10099,Распродажа\nЦена: 300р\nТкань: кожа двухнитка\...,"{""vk_wall_id"": -209013890, ""vk_post_id"": 53366}"


In [86]:
df_filtered_sample.to_pickle('cache/vk_filtered_1500.pkl')